## Load GROQ key

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
groq_api_key = os.getenv('GROQ_API_KEY')

## Initialize model

In [3]:
from langchain_groq import ChatGroq

model =ChatGroq(model="gemma2-9b-it", groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002378D5B3190>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002378D5E0640>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Satish and I am working as a Data Engineer")])

AIMessage(content="Hi Satish, it's nice to meet you!\n\nI'm glad to hear you're a Data Engineer. That's a really interesting field. \n\nWhat kind of projects are you working on these days? \n\nI'm always eager to learn more about what people are doing in the data world.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 24, 'total_tokens': 94, 'completion_time': 0.127272727, 'prompt_time': 0.002117557, 'queue_time': 0.238227721, 'total_time': 0.129390284}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-cb4f145d-5e80-4648-90e0-eff8eb32bd56-0', usage_metadata={'input_tokens': 24, 'output_tokens': 70, 'total_tokens': 94})

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is Satish and  I am working as a Data Engineer"),
        AIMessage(content="Hi Satish, it's nice to meet you!\n\nI'm glad to hear you're a Data Engineer. That's a really interesting field. \n\nWhat kind of projects are you working on these days? \n\nI'm always eager to learn more about what people are doing in the data world.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 24, 'total_tokens': 94, 'completion_time': 0.127272727, 'prompt_time': 0.002117557, 'queue_time': 0.238227721, 'total_time': 0.129390284}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-cb4f145d-5e80-4648-90e0-eff8eb32bd56-0', usage_metadata={'input_tokens': 24, 'output_tokens': 70, 'total_tokens': 94}),
        HumanMessage(content="Hey what's my name and what do I do ?")
    ]
)

AIMessage(content="You said your name is Satish, and you work as a Data Engineer. 😊  \n\nIs there anything else you'd like to tell me about your work?  I'm curious to hear more!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 115, 'total_tokens': 161, 'completion_time': 0.083636364, 'prompt_time': 0.005311202, 'queue_time': 0.23443555700000002, 'total_time': 0.088947566}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-12326a06-e57a-41a4-b1d4-e0fc68c9f972-0', usage_metadata={'input_tokens': 115, 'output_tokens': 46, 'total_tokens': 161})

## Message History
Message history in a chatbot is the record of previous user and bot messages. It helps the chatbot maintain context, understand follow-up questions, and respond more naturally in ongoing conversations.

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory # to tracks messages in memory.
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# In-memory session store
store={}

# Get/Create chat history for a session
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory() # ensures each session ID gets its own message history.
    return store[session_id]

# wraps the model with message history, so every invocation keeps context based on the session.
with_message_history=RunnableWithMessageHistory(model,get_session_history)

## Simulate conversation

In [9]:
session_id = "chat1"
config = {"configurable": {"session_id": session_id}}

In [10]:
# First user message
response1 = with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Satish and I am a Data Engineer")],
    config=config
)

In [12]:
response1.content

"Hi Satish, it's nice to meet you! 👋\n\nBeing a Data Engineer is a really interesting field. What kind of projects are you working on these days?  Anything exciting? 😄  \n\n"

In [13]:
# Second user message (uses previous context automatically)
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Satish!  😊  \n\nI remember that from our introduction.  Is there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 80, 'total_tokens': 111, 'completion_time': 0.056363636, 'prompt_time': 0.004176253, 'queue_time': 0.235540795, 'total_time': 0.060539889}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-58f8b75c-a515-44d7-8ec6-77c691393d6c-0', usage_metadata={'input_tokens': 80, 'output_tokens': 31, 'total_tokens': 111})

In [14]:
# Second user message (uses previous context automatically)
with_message_history.invoke(
    [HumanMessage(content="Can you suggest good projects")],
    config=config,
)

AIMessage(content="Of course, Satish! I can definitely suggest some data engineering projects. \n\nTo give you the best recommendations, tell me:\n\n1. **What's your current skill level?** (Beginner, intermediate, advanced)\n2. **What are you most interested in?** (e.g., real-time data processing, machine learning pipelines, data warehousing, etc.)\n3. **Do you prefer working with specific tools or technologies?** (e.g., Spark, Kafka, Airflow, Snowflake, etc.)\n4. **Do you have a specific domain in mind?** (e.g., finance, healthcare, e-commerce, etc.)\n\nOnce I have this information, I can suggest projects that are both challenging and rewarding for you. \n\n\n\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 164, 'prompt_tokens': 124, 'total_tokens': 288, 'completion_time': 0.298181818, 'prompt_time': 0.006743298, 'queue_time': 0.240107232, 'total_time': 0.304925116}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason

You’ll see the bot remembers the context (e.g., your name and role) in the second message and replies accordingly.

## New Session

In [15]:
# New session
new_session_id = "chat2"
new_config = {"configurable": {"session_id": new_session_id}}

In [16]:
# First message in new session
response_new = with_message_history.invoke(
   [HumanMessage(content="What's my name?")],
    config=new_config
)
response_new.content

"As an AI, I have no access to your personal information, including your name. If you'd like to tell me your name, I'd be happy to know!\n"

In [17]:
response = with_message_history.invoke(
   [HumanMessage(content="What's my name?")],
    config=config
)
response.content

'Your name is Satish!  😊  \n\nI remember that from our earlier conversation.  \n\nIs there anything else I can help you with today, Satish?\n'

- Session chat1: Bot has prior context like your name and role.
- Session chat2: Bot doesn’t know anything yet — clean slate.

In [18]:
response_new = with_message_history.invoke(
   [HumanMessage(content="My name is Kumar and learning Gen AI")],
    config=new_config
)
response_new.content

"It's great to meet you, Kumar! Learning about Generative AI is exciting. \n\nWhat specifically are you interested in learning about? I can try my best to answer your questions or provide you with helpful resources. \n\nFor example, are you curious about:\n\n* **How Gen AI works?**\n* **Different types of Gen AI models (like me!)?**\n* **Applications of Gen AI in various fields?**\n* **The ethical considerations surrounding Gen AI?**\n\n\nLet me know what sparks your interest! 😊\n"

In [19]:
# First message in new session
response_new = with_message_history.invoke(
   [HumanMessage(content="What's my name?")],
    config=new_config
)
response_new.content

'You told me your name is Kumar earlier!  😊 \n\nIs there anything else I can help you with, Kumar?  \n'

## Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

#### Create a prompt template

In [27]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

**What MessagesPlaceholder Does**

It acts as a slot where LangChain injects prior messages (from memory/history) automatically.

This is perfect for open-ended chatbots where you want to maintain back-and-forth context.

In [28]:
# Create chain with output parser
from langchain_core.output_parsers import StrOutputParser
chain=prompt|model|StrOutputParser()


In [29]:
chain.invoke({"messages":[HumanMessage(content="Hi! My name is Satish")]})

"Hello Satish! It's nice to meet you.  \n\nI'm happy to help with any questions you have.  \n\nWhat can I do for you today? 😊  \n\n"

In [30]:
# Wrap chain with message history
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [31]:
# Simulate a conversation
config = {"configurable": {"session_id": "chat03"}}

In [38]:
response1=with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config
)

response1

'Your name is Satish.  \n\nIs there anything else I can help you with?  😄 \n'

#### Prompt with MessagesPlaceholder + language variable

In [45]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Always reply in {language}, no matter what the input language is. Your job is to answer all questions to the best of your ability."),
    MessagesPlaceholder(variable_name="messages"),
    ("human", "{input}")
])

In [46]:
# Chain
chain = prompt | model | StrOutputParser()

# Chain with message history
chat_chain = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",     # for the user input text
    history_messages_key="messages" # this injects prior messages
)

In [49]:
config = {"configurable": {"session_id": "chat04"}}

In [51]:
response1 = chat_chain.invoke(
    {"input": "What is my name?", "language": "TElugu"},
    config=config
)
response1

'నాకు మీ పేరు తెలియదు. మీరు నాకు మీ పేరు చెప్పగలరూ? 😊 \n'

In [52]:
response2 = chat_chain.invoke(
    {
        "input": "My name is Satish staying in Bangalore.",
        "language": "Telugu"
    },
    config=config
)
response2

'నమస్తే శ్రీ సతీష్! బెంగళూరులో నివాసం ఉంటున్నారని తెలిసింది. 😊 \n'

In [54]:

chat_chain.invoke(
    {
        "input": "What is my name and where do I stay?",
        "language": "Telugu"
    },
    config=config
)

'మీ పేరు శ్రీ సతీష్ మరియు బెంగళూరులో నివసిస్తున్నారు. 😊  \n'